<a href="https://colab.research.google.com/github/darkzard05/Cora_dataset_benchmark/blob/main/appnp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch-scatter torch-sparse torch-spline-conv torch_geometric -f https://data.pyg.org/whl/torch-1.11.0+cu113.html

Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 7.9 MB 6.8 MB/s 
     |████████████████████████████████| 3.5 MB 59.9 MB/s 
     |████████████████████████████████| 750 kB 53.3 MB/s 
     |████████████████████████████████| 407 kB 9.2 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=ce73bf16e8806f07b34e2d0832994cd0c4e0ba4c8cfbf997c3e00747462fe67c
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [2]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear
import torch_geometric
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import APPNP

print(torch.__version__)

1.11.0+cu113


In [3]:
dataset_name = 'Cora'
dataset = Planetoid(root='/tmp/'+dataset_name, name=dataset_name)

Processing...
Done!


In [4]:
data = dataset[0]
print(f'dataset name: {dataset_name}')
print(f'number of nodes: {data.num_nodes}, number of edges: {data.num_edges}')
print(f'number of classes: {dataset.num_classes}, number of edge features: {data.num_edge_features}')
print(data.train_mask[:20])
print(data.train_mask.sum(), data.val_mask.sum(), data.test_mask.sum())

dataset name: Cora
number of nodes: 2708, number of edges: 10556
number of classes: 7, number of edge features: 0
tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True])
tensor(140) tensor(500) tensor(1000)


In [5]:
class Model(torch.nn.Module):
    def __init__(self, hidden, K, alpha):
        super().__init__()
        self.layer1 = Linear(dataset.num_features, hidden)
        self.layer2 = Linear(hidden, dataset.num_classes)
        self.prop = APPNP(K=K, alpha=alpha)

    def reset_parameters(self):
        self.layer1.reset_parameters()
        self.layer2.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.dropout(x, training=self.training)
        x = self.layer1(x).relu()
        x = F.dropout(x, training=self.training)
        x = self.layer2(x).relu()
        x = self.prop(x, edge_index)
        return x

In [6]:
print('cuda' if torch.cuda.is_available() else 'cpu', 'is available')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(hidden=16, K=50, alpha=0.1).to(device)
print(model)
model.reset_parameters()
print({i:list(j.shape) for i, j in model.named_parameters()})

data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=5e-3)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    pred = model(data)
    optimizer.zero_grad(set_to_none=True)
    loss = criterion(pred[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

def test(mask):
    model.eval()
    pred = model(data)
    optimizer.zero_grad(set_to_none=True)
    correct = pred.argmax(dim=1)[mask] == data.y[mask]
    acc = int(correct.sum()) / int(mask.sum())
    return acc

best_acc = 0
epochs = 1000
for epoch in range(1, epochs+1):
    loss = train()
    train_acc = test(data.train_mask)
    val_acc = test(data.val_mask)
    test_acc = test(data.test_mask)
    if best_acc < test_acc:
        best_acc = test_acc
        print(f'epoch: {epoch} | loss: {loss.item():.4f} | train_acc: {train_acc:.4f} | val_acc: {val_acc} | test_acc: {test_acc}')

cuda is available
Model(
  (layer1): Linear(in_features=1433, out_features=16, bias=True)
  (layer2): Linear(in_features=16, out_features=7, bias=True)
  (prop): APPNP(K=50, alpha=0.1)
)
{'layer1.weight': [16, 1433], 'layer1.bias': [16], 'layer2.weight': [7, 16], 'layer2.bias': [7]}
epoch: 1 | loss: 1.9522 | train_acc: 0.1429 | val_acc: 0.164 | test_acc: 0.147
epoch: 2 | loss: 1.9361 | train_acc: 0.2714 | val_acc: 0.302 | test_acc: 0.271
epoch: 3 | loss: 1.9197 | train_acc: 0.4214 | val_acc: 0.39 | test_acc: 0.37
epoch: 4 | loss: 1.9000 | train_acc: 0.4143 | val_acc: 0.404 | test_acc: 0.394
epoch: 5 | loss: 1.8851 | train_acc: 0.4571 | val_acc: 0.426 | test_acc: 0.408
epoch: 6 | loss: 1.8625 | train_acc: 0.5000 | val_acc: 0.456 | test_acc: 0.445
epoch: 7 | loss: 1.8305 | train_acc: 0.5143 | val_acc: 0.47 | test_acc: 0.456
epoch: 8 | loss: 1.7999 | train_acc: 0.5214 | val_acc: 0.478 | test_acc: 0.464
epoch: 9 | loss: 1.7855 | train_acc: 0.5286 | val_acc: 0.478 | test_acc: 0.471
epoch: 1